<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<p>
<h1>Algorísmica Avançada</h1>
<h2>Pràctica 2.A - Greedy </h2>
</center>
</p>
</div>

<div class="alert alert-info">
<center>
  <h1>Introducción</h1>
</center>

<p>Un algoritmo Greedy es aquel que trata de resolver un problema con la heurística de escoger, a cada paso, la opción óptima con la intención de encontrar una solución óptima global al problema. En caso general, este tipo de algoritmos no son capaces de encontrar la solución óptima, sin embargo, sí pueden encontrar soluciones subóptimas suficientemente cercanas con una coste computacional significativamente menor.</p>

<h3>Ejemplo</h3>
<p>Supongamos que intentamos encontrar la suma más grande en este grafo en forma de árbol. Un algoritmo greedy no consiste en encontrar una estrategia óptima global al problema, sino que a cada paso, buscará entre sus siguientes opciones cuál lleva más cerca de la solución (óptimo local).</p>
<img src="https://upload.wikimedia.org/wikipedia/commons/8/8c/Greedy-search-path-example.gif">

<h2>Estructura</h2>

Todos los algoritmos greedy comparten ciertas características:
<ol>
    <li><b>Conjunto</b> de elementos a partir de los cuales formar una solución.</li>
    <li>Criterio de <b>elección</b> del siguiente elemento candidato (Heurística).</li>
    <li>Criterio de <b>validación</b> sobre los elementos candidatos.</li>
    <li>Criterio de <b>terminación</b>, que indica cuando hemos alcanzado una solución completa.</li>
    <li>Métrica de <b>evaluación</b> de una solución, total o parcial.</li>
</ol>

<div class="alert alert-info">
<center>
  <h1>Contenido</h1>
  </center><p>



<div class="alert alert-success" style="width:90%; margin:0 auto;">
<h2>0 - Greedy Solver</h2>

Dado que todos los algoritmos Greedy siguen la misma estrategia, es posible crear una función que, dados los criterios de elección, validación y terminación, resuelva cualquier problema genéricamente. Adicionalmente, una función de evaluación de soluciones para determinar lo buena que es la solución propuesta por el algoritmo.

In [1]:
import random
class GreedySolver:
    # Define los criterios que utilizará el GreedySolver
    def __init__(self, choice, is_valid, is_complete, evaluate,preprocess=lambda D: None):
        self._choice = choice
        self._is_valid = is_valid
        self._is_complete = is_complete
        self._evaluate = evaluate
        self._preprocess = preprocess


            
    # Estrategia genérica de resolución de problemas
    def solve(self, D):
        self._preprocess(D)
        x = []
        # Mientras no se cumpla el criterio de terminación
        while D and not self._is_complete(x, D):
            # Escoge un elemento
            d = self._choice(x, D)
            # Comprueba si el elemento puede ser incluido en la solución
            if self._is_valid(x, d):
                x.append(d)

        # Devuelve la solución y la métrica
        return x, self._evaluate(x)

# Genera un conjunto de objetos de prueba
def random_objects(one_or_inf=True):
    objects = []
    # Genera una lista de listas, cada una de las cuales representa un elemento [peso,valor] o [peso,valor,cantidad]
    if one_or_inf:
        for p,v in zip(random.sample(range(1, 50), k=10), random.sample(range(1, 100), k=10)):
            objects.append([p,v])
    else:
        for p,v,c in zip(random.sample(range(1, 50), k=10), random.sample(range(1, 100), k=10),random.sample(range(1, 15), k=10)):
            objects.append([p,v,c])
    return objects

<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>1- El problema de la mochila</p></h2>
  
  <center><img src="img/knapsack.png" width=40%></center>
  
  <p>
    Nos encontramos en una habitación en la que hay $N$ objetos con pesos $w_1, w_2, w_3 ... w_N$ y tenemos una mochila que puede soportar una carga máxima de $W$.
    En esta práctica se pide que realicéis una implementación Greedy para resolver los siguientes problemas:
 <ol><li>
 **mochila:** En este caso buscamos llenar la mochila lo máximo posible. En otras palabras,queremos que $\sum_{i=0}^{N}{(w_i · b_i)}\leq W$ aproximándose lo más posible a $W$, donde $b_k$ es un valor binario que representa si el objeto es escogido o no.
 </li><li>
 **mochila_valor:** Igual que en el caso anterior, pero ahora cada objeto tendrá asignado un valor $v_1, v_2, v_3 ... v_N$. En este caso buscamos llenar la mochila maximizando el valor total de la mochila. Es decir queremos encontrar la combinación de objetos $b$ tal que $\arg_{b} \max{\sum_{i=0}^{N}{v_i · b_i}}$ manteniendo la condición anterior de que $\sum_{i=0}^{N}{(w_i · b_i)}\leq W$.
     </li>
     <li>
     **mochila_diversa:** Para este problema queremos meter en la mochila la mayor cantidad de objetos posibles. De esta manera, buscamos que la combinación de objetos $b$ tal que $\arg_{b} \max{\sum_{i=0}^{N}{b_i}}$.
     </li><li>
     **mochila_fraccionada:** Aquí podemos coger solo una parte del objeto. Así pues, dispondremos de una lista $x_1, x_2, x_3 ... x_N$ tal que $0 \leq x_i \leq 1$ y queremos que $\arg_{b} \max{\sum_{i=0}^{N}{v_i · x_i}}$ manteniendo la condición anterior de que $\sum_{i=0}^{N}{(w_i · x_i)}\leq W$.
     </li>
 </ol>
</p>

<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center>
<p>
**choice.** Esta función debe seleccionar el siguiente elemento.
    <ul>
        <li>Input:
            <ul>
                <li>x. Elementos que forman la solución actual</li>
                <li>D. Conjunto de elementos disponibles</li>
            </ul>
        </li>
        <li>Output:
            <ul>
                <li>d. Elemento seleccionado</li>
            </ul>
        </li>
        </ul>
</p>

<p>
**is_valid.** Esta función debe decidir si el elemento seleccionado puede añadirse a la solución.
    <ul>
        <li>Input:
            <ul>
                <li>x. Elementos que forman la solución actual</li>
                <li>d. Elemento seleccionado</li>
            </ul>
        </li>
        <li>Output:
            <ul>
                <li>valid. Booleano que indica la validez del elemento a añadir</li>
            </ul>
        </li>
    </ul>
</p>

<p>
**is_complete.** Esta función debe decidir si la solución encontrada es completa o si no puede continuarse.
    <ul>
        <li>Input:
            <ul>
                <li>x. Elementos que forman la solución actual</li>
                <li>D. Conjunto de elementos disponibles</li>
            </ul>
        </li>
        <li>Output:
            <ul>
                <li>complete. Booleano que indica si la solución es completa o imposible seguir</li>
            </ul>
        </li>
    </ul>
</p>

<p>
**evaluate.** Esta función devuelve una métrica indicando lo buena que es la solución encontrada.
    <ul>
        <li>Input:
            <ul>
                <li>x. Elementos que forman la solución</li>
            </ul>
        </li>
        <li>Output:
            <ul>
                <li>metric. Valor de la métrica evaluada sobre la solución</li>
            </ul>
        </li>
    </ul>
</p>
<p> <b> Nota: </b> El output de llamar a <i>GreedySolver.solve(D)</i> debe seguir un formato concreto:
    <ul>
        <li> La solución será la lista de objetos escogidos con la forma $[w_i,v_i]$ cuando solo se pueda coger un elemento de cada tipo y con la forma $[w_i,v_i,c_i]$ cuando la cantidad de objetos sea relevante.</li>
        <li> A la solución lo acompañará el resultado de la función <i>GreedySolver.evaluate()</i>: el peso total de la mochila $w_t$ o una tupla $(w_T,v_T)$ para indicar el peso y valor totales de la mochila (en los casos en que el valor sea relevante).</li>
    </ul>
</p>
</div>


Define para cada una de las variantes del problema de la mochila las funciones necesarias para el correcto funcionamiento del GreedySolver.

In [2]:
# Capacidad de la mochila
W = 150

In [3]:
def choice1(x, D):

    return D.pop()
        
        
    

def is_valid1(x, d):
    res=0
    for i in x:
        res=res+i[0]
        
    if res+d[0]>W:
        return False
    elif res+d[0]<=W:
        return True
    
    

def is_complete1(x, D):
    if is_valid1==False:
        return True
    return False
   
    

def evaluate1(x):
    tot=0
    for i in x:
        tot+=i[0]
    return tot

def preprocess1(D):
    D.sort()

    
        

In [4]:
def choice2(x, D):
   
    
    return D.pop()
    
        
def is_valid2(x, d):
    res=0
    for i in x:
        res=res+i[0]
        
    if res+d[0]>W:
        return False
    elif res+d[0]<=W:
        return True
    

def is_complete2(x, D):
    if is_valid2==False:
        return True
    return False

def evaluate2(x):
    tot=0
    val=0
    for i in x:
        tot+=i[0]
        val+=i[1]
    return tot,val
def preprocess2(D):
    D.sort(key=lambda x: x[1]/x[0])


In [5]:
def choice3(x, D):
    
    return D.pop(0)
    

def is_valid3(x, d):
    res=0
    for i in x:
        res=res+i[0]
        
    if res+d[0]>W:
        return False
    elif res+d[0]<=W:
        return True
    

def is_complete3(x, D):
    if is_valid3==False:
        return True
    return False

def evaluate3(x):
    tot=0
    for i in x:
        tot+=i[0]
    return tot
def preprocess3(D):
    D.sort()

In [6]:
def choice4(x, D):
    
    
    return D.pop()
        
        

def is_valid4(x, d):
    
    res=0
    for i in x:
        res=res+i[0]
    if res+d[0]<=W:
        
        return True
    
    elif res+d[0]>W:
        if res==W:
            return False
        elif res!=W:
            fraccion=W-res
            peso_tmp,valor_tmp=d[0],d[1]
            if d[0]>fraccion:
                d[0]=fraccion
                d[1]=(fraccion*valor_tmp)/peso_tmp
        return True
        
        
    return False
    
    

def is_complete4(x, D):
    if is_valid4==False:
        
        return True
        
    return False

def evaluate4(x):
    tot=0
    val=0
    for i in x:
        tot+=i[0]
        val+=i[1]
    return tot,val
def preprocess4(D):
    D.sort(key=lambda x: x[1]/x[0])


Añade aquí el código para instanciar un GreedySolver y utilizarlo para resolver cada uno de los problemas y mostrar la solución y su evaluación en pantalla.

In [7]:
import copy
# Con un seed fijo, el generador de objetos no cambia
random.seed(0)

# Genera una lista de elementos al azar en forma listas [peso, valor]
D = random_objects()

print(D)
# Instancia los GreedySolvers y úsalos para resolver las variantes del problema de la mochila
G1 = GreedySolver(choice1,is_valid1,is_complete1,evaluate1,preprocess1)
print(G1.solve(copy.deepcopy(D)))
G2 = GreedySolver(choice2,is_valid2,is_complete2,evaluate2,preprocess2)
print(G2.solve(copy.deepcopy(D)))
G3 = GreedySolver(choice3,is_valid3,is_complete3,evaluate3,preprocess3)
print(G3.solve(copy.deepcopy(D)))
G4 = GreedySolver(choice4,is_valid4,is_complete4,evaluate4,preprocess4)
print(G4.solve(copy.deepcopy(D)))

[[25, 75], [27, 28], [3, 65], [17, 18], [33, 37], [32, 97], [26, 13], [20, 80], [31, 33], [23, 69]]
([[33, 37], [32, 97], [31, 33], [27, 28], [26, 13]], 149)
([[3, 65], [20, 80], [32, 97], [23, 69], [25, 75], [33, 37]], (136, 423))
([[3, 65], [17, 18], [20, 80], [23, 69], [25, 75], [26, 13], [27, 28]], 141)
([[3, 65], [20, 80], [32, 97], [23, 69], [25, 75], [33, 37], [14, 14.903225806451612]], (150, 437.9032258064516))


<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h3>1.1- La mochila con varios objetos</h3>
  <p>Supongamos ahora que en lugar de tener un único elemento de cada objeto, tenemos una cantidad determinada de cada uno y la solución no solo consiste en coger el objeto o no, sino también en cuántos cogemos de cada objeto. ¿Cómo cambia la implementación de la solución del ejercicio 2? Impleméntalo. ¿Es posible resolverlo con el esquema dado del GreedySolver? En caso negativo, crea una nueva clase GreedySolver_v con los cambios que consideres oportunos.
</p>

In [8]:
class GreedySolver_v:
    # Define los criterios que utilizará el GreedySolver
    def __init__(self, choice, is_valid, is_complete, evaluate,preprocess=lambda D: None):
        self._choice = choice
        self._is_valid = is_valid
        self._is_complete = is_complete
        self._evaluate = evaluate
        self._preprocess = preprocess 


    # Estrategia genérica de resolución de problemas
    def solve(self, D):
        self._preprocess(D)
        x = []
        # Mientras no se cumpla el criterio de terminación
        while D and self._is_complete(x, D)==False:
            # Escoge un elemento
            
            origen = self._choice(x, D)
            d=D[origen]
            # Comprueba si el elemento puede ser incluido en la solución
            #Compruebo si es valido y ademas si aún nos quedan objetos de ese tipo
            if self._is_valid(x,d) and d[2]>0:
                x.append(d)
                d[2]-=1
            else:
                #si no quedan, lo eliminamos de la lista
                D.pop()
                

        # Devuelve la solución y la métrica
        return self._evaluate(x)

# Genera un conjunto de objetos de prueba
def random_objects(one_or_inf=True):
    objects = []
    # Genera una lista de listas, cada una de las cuales representa un elemento [peso,valor] o [peso,valor,cantidad]
    if one_or_inf:
        for p,v in zip(random.sample(range(1, 50), k=10), random.sample(range(1, 100), k=10)):
            objects.append([p,v])
    else:
        for p,v,c in zip(random.sample(range(1, 50), k=10), random.sample(range(1, 100), k=10),random.sample(range(1, 15), k=10)):
            objects.append([p,v,c])
    return objects

In [9]:
def choice2_v(x, D):
    origen=0
    left_or=0
    right_or=0
    
    if len(D)!=1:
        
        if D[0][1]/D[0][0]<D[1][1]/D[1][0]:
            right_or=1
        if D[0][1]/D[0][0]<D[-1][1]/D[-1][0]:
            left_or=-1
        if D[right_or][1]/D[right_or][0]>D[left_or][1]/D[left_or][0]:
            origen=right_or
        else:
            origen=left_or
    
    return origen
    
    

def is_valid2_v(x, d):
    res=0
    for i in x:
        res=res+i[0]
    if res+d[0]>W:
        return False
    elif res+d[0]<=W:
        return True
    

def is_complete2_v(x, D):
    
    if is_valid2_v==False:
        return True
    return False
    

def evaluate2_v(x):
    tot=0
    val=0
    x_real=[]
    for i in x:
        
        tot+=i[0]
        val+=i[1]
    print(x)
    count=0
    for i in range (0,len(x)):
        if i<len(x)-1:
            if x[i][0]==x[i+1][0]:
                count+=1
            elif x[i][0]!=x[i+1][0]:
            
                x[i][2]=count
                x_real.append(x[i])
                count=0
        else:
            if x[-1][0]==x[i][0]:
                count+=1
            
        
            
    x[i][2]=count
    x_real.append(x[i])
    count=0
            

            
    
        
    
    
    return x_real,(tot,val)


def preprocess2_v(D):
    D.sort(key=lambda x: x[1]/x[0])


In [10]:
import random
random.seed(0)

# Genera una lista de elementos al azar en forma de listas [peso, valor, cantidad]
D=random_objects(False)
print(D)

# Instancia el GreedySolver y úsalo para resolver esta variante del problema de la mochila
Gv2 = GreedySolver_v(choice2_v,is_valid2_v,is_complete2_v,evaluate2_v,preprocess2_v)
print(Gv2.solve(copy.deepcopy(D)))

[[25, 75, 12], [27, 28, 13], [3, 65, 10], [17, 18, 3], [33, 37, 5], [32, 97, 2], [26, 13, 9], [20, 80, 7], [31, 33, 6], [23, 69, 11]]
[[3, 65, 0], [3, 65, 0], [3, 65, 0], [3, 65, 0], [3, 65, 0], [3, 65, 0], [3, 65, 0], [3, 65, 0], [3, 65, 0], [3, 65, 0], [20, 80, 1], [20, 80, 1], [20, 80, 1], [20, 80, 1], [20, 80, 1], [20, 80, 1]]
([[3, 65, 9], [20, 80, 6]], (150, 1130))


<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h3>1.2- La mochila con infinitos objetos</h3>
<p>Supongamos ahora que tenemos una cantidad ilimitada de cada objeto. ¿Cómo cambia la implementación de la solución del ejercicio ahora? Impleméntalo. ¿Es posible resolverlo con el esquema dado del GreedySolver? En caso negativo, crea una nueva clase GreedySolver_inf con los cambios que consideres oportunos.
</p>

In [11]:
class GreedySolver_inf:
    # Define los criterios que utilizará el GreedySolver
    def __init__(self, choice, is_valid, is_complete, evaluate,preprocess=lambda D: None):
        self._choice = choice
        self._is_valid = is_valid
        self._is_complete = is_complete
        self._evaluate = evaluate
        self._preprocess = preprocess 

    # Estrategia genérica de resolución de problemas
    def solve(self, D):
        self._preprocess(D)
        x = []
        # Mientras no se cumpla el criterio de terminación
        while D and not self._is_complete(x, D):
            # Escoge un elemento
            origen = self._choice(x, D)
            d=D[origen]
            # Comprueba si el elemento puede ser incluido en la solución
            if self._is_valid(x, d):
                   
                x.append(d)
            else:
                #Aqui vamos a eliminar el elemento que no cumple los requisitos, por ejemplo, que no cabe.
                D.pop(origen)

        # Devuelve la solución y la métrica
        return self._evaluate(x)

# Genera un conjunto de objetos de prueba
def random_objects(one_or_inf=True):
    objects = []
    # Genera una lista de listas, cada una de las cuales representa un elemento [peso,valor] o [peso,valor,cantidad]
    if one_or_inf:
        for p,v in zip(random.sample(range(1, 50), k=10), random.sample(range(1, 100), k=10)):
            objects.append([p,v])
    else:
        for p,v,c in zip(random.sample(range(1, 50), k=10), random.sample(range(1, 100), k=10),random.sample(range(1, 15), k=10)):
            objects.append([p,v,c])
    return objects

In [12]:
def choice2_inf(x, D):
    origen=0
    left_or=0
    right_or=0
    
    if len(D)!=1:
        
        if D[0][1]/D[0][0]<D[1][1]/D[1][0]:
            right_or=1
        if D[0][1]/D[0][0]<D[-1][1]/D[-1][0]:
            left_or=-1
        if D[right_or][1]/D[right_or][0]>D[left_or][1]/D[left_or][0]:
            origen=right_or
        else:
            origen=left_or
    
    return origen
    

def is_valid2_inf(x, d):
    res=0
    for i in x:
        res=res+i[0]

    if res+d[0]<=W:
        return True
    elif res+d[0]>W:
        return False
    
    
    

def is_complete2_inf(x, D):
    if is_valid2_inf==False:
        return True
    return False
    

def evaluate2_inf(x):
    x_real=[]
    tot=0
    val=0
    for i in x:
        tot+=i[0]
        val+=i[1]
    count=0
    for i in range (0,len(x)):
        if i<len(x)-1:
            if x[i][0]==x[i+1][0]:
                count+=1
            elif x[i][0]!=x[i+1][0]:
            
                x[i][2]=count
                x_real.append(x[i])
                count=0
        else:
            if x[-1][0]==x[i][0]:
                count+=1
            
        
            
    x[i][2]=count
    x_real.append(x[i])
    count=0
    
    return x_real,(tot,val)
def preprocess2_inf(D):
    D.sort(key=lambda x: x[1]/x[0])


In [15]:
import random
random.seed(0)

# Genera una lista de elementos al azar en forma listas [peso, valor]
D = random_objects(False)
print(D)


# Instancia el GreedySolver y úsalo para resolver esta variante del problema de la mochila
G2_inf = GreedySolver_inf(choice2_inf,is_valid2_inf,is_complete2_inf,evaluate2_inf,preprocess2_inf)
print(G2_inf.solve(copy.deepcopy(D)))

[[25, 75, 12], [27, 28, 13], [3, 65, 10], [17, 18, 3], [33, 37, 5], [32, 97, 2], [26, 13, 9], [20, 80, 7], [31, 33, 6], [23, 69, 11]]
([[3, 65, 50]], (150, 3250))


<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Cuestiones</h3></p> </center> </div>

<ul>
    <li>¿En qué casos se encuentra solución óptima al problema?</li>
    <li>Explica las soluciones planteadas y analiza su complejidad.</li>
    <li>Explica la estrategia que has empleado para las variantes del problema 2, enfatizando las diferencias principales con respecto al problema original ¿Ha cambiado la complejidad? Justifica tu respuesta.</li>
</ul>

__Escribe aquí tus respuestas__

<h4> Pregunta 1 </h4>

La solución óptima se encuentra cuando ordenamos los elementos por su peso, o su ratio de peso y valor. De esta forma, no debemos recorrer la lista múltiples veces para encontrar el elemento que se adecua más a nuestra siguiente elección.



<h4> Pregunta 2 </h4>

En el 1 y el 3 he decidido ordenar de mayor a menor y de menor a mayor respectivamente, de esa manera podremos escoger el mas adecuado a cada iteración.
En el 2 y en el 4 he ordenado de mejor a peor ratio de valor/precio. La diferencia se encuentra en el 4, que cuando no nos cabe nada más, cogemos una fracción del elemento con mejor ratio en ese momento.

En los dos últimos he ordenado por mejor ratio y ir escogiendo hasta que fuera permitido o válido.

La complejidad de este problema es O(n log n)




<h4> Pregunta 3 </h4>

Para el problema dos he empezado ordenando por mejor ratio en ambos.

-Si nos centramos para el problema de limitados, he devolviendo el primero, en forma de indice (un entero) y cuando este no era valido o su numero de unidades estaba a 0 hacia un pop del elemento. Hacer el pop reduce la complejidad, ya que remove es más costoso.

-Por otra parte, en el de objetos infinitos, tambien he devuelto el primero, en forma de indicie (entero) como en el anterior caso, pero la condición era que si el elemento no era válido pues haciamos pop.

Las principales diferencias con el problema del principio son que en uno podiamos hacer un pop de la lista y de esa forma eliminarlo. En las dos variantes del 2, he modificado el GS para poder usar mi implementación del indice.


<h4> Pregunta 4 </h4>
La complejidad es la misma, solamente que al tener varios objetos, debemos tratar el problema de diferente forma.

<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>
<p>
La entrega de esta práctica (junto a la de programación dinámica) se podrá realizar en el campus virtual hasta el día __25 de Noviembre a las 23:55__. En la tarea que se habilitará en el campus deberéis colgar __un archivo .zip__ que contenga __únicamente__ los notebooks de las dos prácticas siguiendo la siguiente nomenclatura:
</p>
<p>
```
*AA_P2_<apellido1>_<apellido2>_<nombre>.zip
|___* AA_Greedy_<apellido1>_<apellido2>_<nombre>.ipynb
|___* AA_PD_<apellido1>_<apellido2>_<nombre>.ipynb
```

</p>
<p>
Por ejemplo:
</p>
<p>
```
AA_P2_Doe_Ritchie_John.zip
```

</p>
<p>

Es fundamental que el código esté bién comentado y con un análisis de complejidad exhaustivo del algoritmo.También resulta de gran importancia que la entrega siga el formato indicado. 
</p>
</div>